In [1]:
!wget https://raw.githubusercontent.com/Neoanarika/MBTI/master/mbti_1.csv

--2021-04-30 08:48:40--  https://raw.githubusercontent.com/Neoanarika/MBTI/master/mbti_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62856486 (60M) [text/plain]
Saving to: ‘mbti_1.csv’

mbti_1.csv          100%[===================>]  59.94M   297MB/s    in 0.2s    

2021-04-30 08:48:41 (297 MB/s) - ‘mbti_1.csv’ saved [62856486/62856486]



In [2]:
!pip install torchtext==0.8.0 transformers

     |████████████████████████████████| 7.0MB 23.5MB/s 
     |████████████████████████████████| 2.1MB 47.4MB/s 
     |████████████████████████████████| 901kB 50.3MB/s 
     |████████████████████████████████| 3.3MB 47.9MB/s 
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


In [4]:
# run this block twice, it throws an error first time, this happens because some stuff in torchtext is being moved to legacy right now
from torchtext.data import Field, TabularDataset, BucketIterator, LabelField
!export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64

In [5]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torchtext.data import Field, TabularDataset, BucketIterator, LabelField

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('mbti_1.csv')
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [8]:
len(df)

8675

In [9]:
labels = df['type'].values.tolist()
num_labels = np.unique(labels)
num_labels, len(num_labels)

(array(['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP',
        'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP'],
       dtype='<U4'), 16)

### preprocessing starts here

In [10]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

max_input_len = tokenizer.max_model_input_sizes["bert-base-uncased"]
def tokenize_and_cut(text):
    tokens = tokenizer.tokenize(text)[:max_input_len - 2]
    return tokens

In [11]:
TEXT = Field(batch_first=True, use_vocab=False, tokenize=tokenize_and_cut, preprocessing=tokenizer.convert_tokens_to_ids,
             init_token=init_token_idx, eos_token=eos_token_idx, pad_token=pad_token_idx, unk_token=unk_token_idx)
LABEL = LabelField(dtype=torch.long)
fields = {"posts": ("posts", TEXT), "type": ("type", LABEL)}

In [12]:
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import pandas as pd
df=pd.read_csv("mbti_1.csv")
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['type'])
df['type'] = le.transform(df['type'])
df.head()

,type,posts
0,8,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,3,'I'm finding the lack of me in these posts ver...
2,11,'Good one _____ https://www.youtube.com/wat...
3,10,"'Dear INTP, I enjoyed our conversation the o..."
4,2,'You're fired.|||That's another silly misconce...


In [13]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy=train_test_split(df['posts'],df['type'],test_size=0.2)

In [14]:
traindf=pd.DataFrame(trainx,columns=['posts'])
traindf['type']=trainy#.astype(float)
#traindf.columns=['text','labels']
testdf=pd.DataFrame(testx,columns=['posts'])
testdf['type']=testy#.astype(float)
#testdf.columns=['text','labels']
traindf.to_csv("train_mbti.csv")
testdf.to_csv("test_mbti.csv")
traindf.head()

,posts,type
4552,'I tell you what's the cutest. lol. He almost...,ISFP
1236,"Hi! didn't read down the whole post, only the ...",ENFP
461,'That certainty of how life is supposed to be....,INTJ
7942,'I've traveled alone and found it for the most...,INFP
2431,'Very nice stuff Winchie! :D I like :) (Altho...,ESFJ


In [15]:
# this block of code will take ~7 mins to run because of the bert tokenizer, use nltk or spacy tokenizer here if u want
train_data, test_data = TabularDataset.splits(
    path="", train="train_mbti.csv", test="test_mbti.csv", format="csv", fields=fields # TabularDataset requires 'test' argument too, here i don't have test dataset so i used train_data as the test_data, use the test data if you have any
)
LABEL.build_vocab(train_data)
train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), batch_size=128, device="cuda" # change the batch_size if u want
)

In [16]:
print(len(LABEL.vocab), LABEL.vocab.stoi, LABEL.vocab.itos)

16 defaultdict(None, {'INFP': 0, 'INFJ': 1, 'INTP': 2, 'INTJ': 3, 'ENFP': 4, 'ENTP': 5, 'ISTP': 6, 'ISFP': 7, 'ENTJ': 8, 'ISTJ': 9, 'ENFJ': 10, 'ISFJ': 11, 'ESTP': 12, 'ESFP': 13, 'ESFJ': 14, 'ESTJ': 15}) ['INFP', 'INFJ', 'INTP', 'INTJ', 'ENFP', 'ENTP', 'ISTP', 'ISFP', 'ENTJ', 'ISTJ', 'ENFJ', 'ISFJ', 'ESTP', 'ESFP', 'ESFJ', 'ESTJ']


In [17]:
for d in train_data:
    print(vars(d)["posts"])
    print(vars(d)["type"])
    break

[1005, 1045, 2425, 2017, 2054, 1005, 1055, 1996, 10140, 3367, 1012, 8840, 2140, 1012, 2002, 2471, 7349, 2033, 27594, 2102, 2833, 1999, 2010, 16716, 2522, 2015, 2002, 2439, 2650, 1997, 2054, 2001, 1999, 2010, 16716, 1012, 1045, 4081, 2000, 5860, 4232, 1996, 13735, 8288, 1010, 1004, 2129, 2055, 1996, 7967, 2015, 2029, 1012, 1012, 1012, 1064, 1064, 1064, 9765, 2361, 2064, 2022, 2061, 2204, 2000, 3071, 1012, 2026, 9765, 2361, 17021, 2015, 1998, 4067, 2296, 3902, 4062, 1010, 2296, 15610, 1010, 4385, 1012, 1045, 2196, 2130, 5060, 2037, 3739, 999, 1045, 1005, 1049, 2061, 7622, 1012, 9805, 2361, 1010, 2026, 9765, 2361, 3662, 2010, 18130, 2000, 2033, 1012, 1012, 1012, 1064, 1064, 1064, 1045, 2031, 2356, 2008, 1010, 2002, 2106, 5466, 1996, 3160, 2067, 2000, 2033, 1012, 1998, 2009, 2001, 2062, 2066, 1010, 2002, 2359, 2033, 2000, 2156, 2005, 2870, 1010, 2054, 2009, 2003, 1012, 1998, 2057, 19030, 2046, 2010, 8628, 1998, 2002, 26979, 2066, 1045, 2001, 2010, 1012, 1012, 1012, 1064, 1064, 1064, 2002, 

In [18]:
import numpy as np
np.unique(trainy,return_counts=True)

(array(['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP',
        'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP'],
       dtype=object),
 array([ 149,  564,  197,  550,   30,   41,   29,   77, 1150, 1470,  872,
        1044,  133,  208,  167,  259]))

In [19]:
batch = next(iter(train_iterator))
print(batch.posts.shape, batch.type.shape, batch.posts, batch.type)

torch.Size([128, 512]) torch.Size([128]) tensor([[  101,  1005,  6077,  ...,  2005,  2151,   102],
        [  101,  1045,  2031,  ...,  1998,  1996,   102],
        [  101,  1005, 16021,  ...,  2097,  2031,   102],
        ...,
        [  101,  1005,  2111,  ...,  2033,  3788,   102],
        [  101,  1005,  2053,  ...,  2023,  2003,   102],
        [  101,  1005,  1045,  ...,  1027,  1039,   102]], device='cuda:0') tensor([ 1,  0,  5,  2,  5,  5,  2,  0,  2,  2, 14,  7,  5,  8,  4,  3,  1,  0,
         0, 10,  3,  0,  5,  3,  0, 12,  9,  4,  9,  3,  0,  0,  1,  0,  4,  6,
         2,  0,  1,  1, 10,  9,  1,  2,  1,  2,  1,  8,  2,  8,  9, 11,  0,  0,
         1, 12,  6,  0, 10,  3,  1,  4,  5,  6,  1,  4,  0,  6,  4,  1,  0,  0,
         9,  3,  7,  8,  4,  2,  3,  5,  7,  2,  0,  4,  0,  8,  0,  3,  0,  3,
         1,  7,  7, 10,  1,  5,  0,  3,  1,  0, 13,  1,  0,  2,  1,  3,  2, 11,
         0,  6,  3,  0,  0,  0,  1,  5,  2,  1,  0,  1,  3,  2,  0,  1,  3,  2,
         6,  0], dev

In [ ]:
train_data

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 10
batch_size = 128
hidden_size = 256
output_size = 16

In [21]:
bert = BertModel.from_pretrained("bert-base-uncased")
class Net(nn.Module):
    def __init__(self, bert, hidden_size, output_size):
        super().__init__()
        self.bert = bert
        embedding_size = bert.config.to_dict()["hidden_size"]

        self.gru = nn.GRU(embedding_size, hidden_size, num_layers=2, bidirectional=True, batch_first=True, dropout=0.25)
        self.fc = nn.Linear(2 * hidden_size, output_size)

        self.dropout = nn.Dropout(0.25)

    def forward(self, text):
        with torch.no_grad():
            embedded = self.bert(text)[0]

        _, hidden = self.gru(embedded)
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        output = self.fc(hidden)

        return output

INFO:filelock:Lock 140218324228880 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock


INFO:filelock:Lock 140218324228880 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
INFO:filelock:Lock 140218311453392 acquired on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock


INFO:filelock:Lock 140218311453392 released on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock


In [22]:
net = Net(bert, hidden_size, output_size).to(device)
net

Net(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        

In [23]:

for name, param in net.named_parameters():
    if "bert" in name:
        param.requires_grad = False

In [24]:

opt = torch.optim.Adam(net.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [29]:
'''def get_accuracy(preds, y):
    preds = torch.round(torch.sigmoid(preds))
    correct = (preds == y).float()
    acc = correct.sum() / len(correct)

    return acc'''

def get_accuracy(preds, y):
    preds = preds.argmax(dim=1, keepdim=True)
    correct = preds.squeeze(1).eq(y)
    acc = correct.sum() / torch.FloatTensor([y.shape[0]]).to(device)
    return acc.item()

def loop(net, batches, train):
    batch_losses = []
    batch_accs = []

    if train:
        print("Train Loop:")
        net.train()
        for batch in tqdm.tqdm(batches, total=len(batches)):
            texts = batch.posts.to(device)
            labels = batch.type.to(device)

            preds = net(texts)
            #preds = preds.squeeze(1)

            loss = loss_fn(preds, labels)
            acc = get_accuracy(preds, labels)

            opt.zero_grad()
            loss.backward()
            opt.step()

            batch_losses.append(loss.item())
            batch_accs.append(acc)

    else:
        print("Inference Loop:")
        net.eval()
        with torch.no_grad():
            for batch in tqdm.tqdm(batches, total=len(batches)):
                texts = batch.posts.to(device)
                labels = batch.type.to(device)

                preds = net(texts)
                preds = preds.squeeze(1)

                loss = loss_fn(preds, labels)
                acc = get_accuracy(preds, labels)

                batch_losses.append(loss.item())
                batch_accs.append(acc) 

    print("")
    print("")
    
    return sum(batch_losses) / len(batch_losses), sum(batch_accs) / len(batch_accs)
'''def predict_sentiment(net, text):
    net.eval()
    tokens = tokenizer.tokenize(text)[:max_input_len - 2]
    indices = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    indices = torch.LongTensor(indices).unsqueeze(0).to(device)
    
    preds = net(indices)
    preds = torch.sigmoid(preds)
    
    print(f"sentiment: {preds.item()}")'''
@torch.no_grad()
def predict(net, text):
    net.eval()
    tokens = tokenizer.tokenize(text)[:max_input_len - 2]
    indices = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    indices = torch.LongTensor(indices).unsqueeze(0).to(device)
    preds = net(indices).argmax(-1)
    preds = LABEL.vocab.itos[preds.item()]
    print(preds)
def predict_return(net, text):
    net.eval()
    tokens = tokenizer.tokenize(text)[:max_input_len - 2]
    indices = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    indices = torch.LongTensor(indices).unsqueeze(0).to(device)
    preds = net(indices).argmax(-1)
    preds = LABEL.vocab.itos[preds.item()]
    return (preds)

In [26]:
text=testx[testx.index[0]]
print(testy[testx.index[0]])

ESTP


In [27]:
#from torchtext import data
import tqdm
train_batches, test_batches = BucketIterator.splits((train_data, test_data), batch_size=batch_size, sort=False,
                                                         device=device)
for epoch in range(epochs):
    train_loss, train_acc = loop(net, train_batches, True)
    val_loss, val_acc = loop(net, test_batches, False)
    
    print(f"epoch: {epoch} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} | val_acc: {val_acc:.4f}")
    predict(net, text)
    print("")

  0%|          | 0/55 [00:00<?, ?it/s]

Train Loop:


  0%|          | 0/14 [00:00<?, ?it/s]



Inference Loop:


  0%|          | 0/55 [00:00<?, ?it/s]

...



epoch: 8 | train_loss: 0.7373 | train_acc: 0.7614 | val_loss: 2.0383 | val_acc: 0.4584
ENTJ

Train Loop:


  0%|          | 0/14 [00:00<?, ?it/s]



Inference Loop:


100%|██████████| 14/14 [01:14<00:00,  5.29s/it]



epoch: 9 | train_loss: 0.4597 | train_acc: 0.8530 | val_loss: 2.2675 | val_acc: 0.4668
ENTJ



In [30]:
c=0
for i in testdf.index:
    if predict_return(net,testdf['posts'][i])==testdf['type'][i]:
        c+=1
print(c,len(testdf))

814 1735


In [31]:
(814/1735)*100

46.9164265129683